# Let us fix margin!!!

In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_covers.pkl', 'df_ohlcs.pkl',
       'df_opt_margins.pkl', 'df_opt_prices.pkl', 'df_symlots.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl'],
      dtype='<U18')

In [27]:
# * IMPORTS
from support import quick_pf, get_openorders
from engine import qualify, get_margins, executeAsync, save_df, margin
from ib_insync import MarketOrder

In [28]:
%%time
with IB().connect(ibp.HOST, ibp.PORT, ibp.CID) as ib:
    df_pf = quick_pf(ib)

Wall time: 1.48 s


,conId,symbol,secType,expiry,strike,right,orderId,permId,action,totalQuantity,lmtPrice,status
0,457634514,BAC,OPT,20210108,20.0,P,1376990,716493364,SELL,1.0,13.82,Submitted
1,455251723,ABC,OPT,20201231,80.0,P,1376991,716493365,SELL,1.0,14.11,Submitted
2,456051730,AAPL,OPT,20201224,65.0,P,1376992,716493366,SELL,1.0,14.53,Submitted
3,458036947,AAPL,OPT,20210108,80.0,P,1376996,716493367,SELL,1.0,15.96,Submitted
4,452872059,GILD,OPT,20201211,50.0,P,1376987,716493360,SELL,1.0,13.46,Submitted
...,...,...,...,...,...,...,...,...,...,...,...,...
671,455260362,DIS,OPT,20201231,100.0,P,1376762,716493135,SELL,1.0,1.62,Submitted
672,455639107,NSC,OPT,20201204,290.0,C,1376754,716493128,SELL,1.0,1.56,Submitted
673,451519615,KR,OPT,20201204,39.0,C,1376756,716493129,SELL,1.0,1.58,Submitted
674,458050003,KLAC,OPT,20210108,180.0,P,1376758,716493130,SELL,1.0,2.81,Submitted


In [9]:
%%time
if not df_pf.empty:
    
    pf_raw_cts = [Contract(conId=c) for c in df_pf.conId]

    with IB().connect(HOST, PORT, CID) as ib:
        pf_cts = ib.run(qualify(ib, pf_raw_cts))

Wall time: 1.52 s


In [10]:
df = pd.concat([pf_cts, pd.Series([MarketOrder('SELL', 1)]*len(pf_cts), name='order')], axis=1)

In [11]:
async def wifAsync(ct, o):
    wif = ib.whatIfOrderAsync(ct, o)
    await asyncio.sleep(FILL_DELAY)
    return wif

In [12]:
co = list(zip(df.contract, df.order))[0]
c, o = co

In [24]:
with IB().connect(HOST, PORT, CID) as ib:
    y = ib.whatIfOrder(c, o)
#     FILL_DELAY = 2
#     wif = ib.run(wifAsync(c, o))

In [25]:
y

OrderState(status='PreSubmitted', initMarginBefore='84673.48', maintMarginBefore='77471.27', equityWithLoanBefore='485654.41', initMarginChange='-4.509999999994761', maintMarginChange='-5.960000000006403', equityWithLoanChange='-0.34999999997671694', initMarginAfter='84668.97', maintMarginAfter='77465.31', equityWithLoanAfter='485654.06', commission=1.7976931348623157e+308, minCommission=0.3482494, maxCommission=0.3544494, commissionCurrency='USD', warningText='', completedTime='', completedStatus='')

In [ ]:
pf_cts[0]

In [ ]:
df_pf = df_pf.assign(contract=pf_cts)
df_pf = df_pf.assign(order=\
                     [MarketOrder("SELL", abs(p)) \
                        if p > 0 \
                          else MarketOrder("BUY", abs(p))\
                        for p in df_pf.position])

In [ ]:
cos = [(c, o) for c, o in zip(df_pf.contract, df_pf.order)]

In [ ]:
co = df_pf.contract[0], MarketOrder(action='SELL', totalQuantity=400.0)

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    y = ib.run(margin(ib, co=co))

In [ ]:
TEMPL_PATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "template",
                                         "df_margin.pkl")
df_empty = pd.read_pickle(TEMPL_PATH)
df_empty

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    df = ib.run(
        executeAsync(
            ib=ib,
            algo=margin,
            cts=cos[:2],
            CONCURRENT=200,
            TIMEOUT=5.5,
            post_process=save_df,
            DATAPATH=DATAPATH,
            OP_FILENAME='',
            SHOW_TQDM=True,
            **{"FILL_DELAY": 5.5},
        ))

In [ ]:
df_pfm = get_margins(MARKET=MARKET,
                     cos=cos)